In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import scipy.integrate
import astropy.units as u
import GCRCatalogs
import pandas as pd

%matplotlib inline

In [ ]:
try:
    import gi

    gi.require_version("NumCosmo", "1.0")
    gi.require_version("NumCosmoMath", "1.0")
except:
    pass

import sys
import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

# Extract DM haloes from the SkySim catalog in a given mass and redshift range. 

## Get catalog information: cosmology, sky area and define cuts

In [ ]:
catalog = "skysim5000_v1.1.1"
skysim_cat = GCRCatalogs.load_catalog(catalog)
cosmo_ss = skysim_cat.cosmology
# skysim_cat.list_all_quantities()

# let's see what masses are availble
print(sorted(c for c in skysim_cat.list_all_quantities(True) if "mass" in c.lower()))
sorted(c for c in skysim_cat.list_all_quantities(True) if "redshift" in c.lower())

In [ ]:
print(cosmo_ss)
print(skysim_cat.sky_area)
print(cosmo_ss.n_s)
print(cosmo_ss.Ode0)
# masses = pd.DataFrame(masses)
# masses['baseDC2/sod_halo_mass/h'] = masses['baseDC2/sod_halo_mass']/cosmo_ss.h
# display(masses[filt])

In [ ]:
zmin = 0.0
zmax = 1.0
mmin = 1.0e14  # Msun.
mmax = 1.0e15  # Msun

ahash = hash((catalog, zmin, zmax, mmin, mmax))
print(ahash)
# ahash =-5265039608222506441

## Perform extraction
**NB: This may be skipped if using directly the files in the `data` directory**

In [ ]:
# Get list of halos in a given redshift and mass range.
# Can only filter on 'halo_mass' which corresponds to the fof mass (in Msun).
# Will extract all haloes with halo_mass > mmin_extract now
# and refine the mass range on M200c, afterwards

"""
mmin_extract = 1.e12 # Msun (M_fof)
dm_halos = skysim_cat.get_quantities(['halo_mass','redshift','baseDC2/sod_halo_mass', 'baseDC2/redshift','baseDC2/sod_halo_cdelta'],
                                     filters=[f'halo_mass > {mmin_extract}','is_central==True',
                                              f'redshift_true>0.0', f'redshift_true<1.5'])


N_cl = len(dm_halos['halo_mass'])
print(f'There are {N_cl} halos in this mass (Mfof) and redshift range')
"""

In [ ]:
from astropy.io import fits
from astropy.table import Table


ncdata_fits = fits.open("skysim5000_halos_m200c_13.fits")
# ncdata_fits.info()

ncdata_data = ncdata_fits[1].data

ncdata_Table = Table(ncdata_data)
ncdata_Table
ncdata_Table["M200c"] = (
    ncdata_Table["baseDC2/sod_halo_mass"] / cosmo_ss.h
)  # sod_halo_mass = M200,c in Msun/h, needs conversion

filt3 = ncdata_Table["M200c"] >= mmin
filt4 = ncdata_Table["M200c"] < mmax
filt5 = ncdata_Table["redshift_true"] >= zmin
filt6 = ncdata_Table["redshift_true"] < zmax

filt = filt3 * filt4 * filt5 * filt6

data_m_ss = ncdata_Table["M200c"][filt]  # M200,c [Msun]
data_z_ss = ncdata_Table["redshift_true"][filt]
N_cl = len(data_m_ss)
print(f"There are {N_cl} halos in this mass (Msod) and redshift range")

### Define a redshift and mass range
NB: SkySim5000 M200c masses are in units of Msun/h

In [ ]:
# dm_halos['M200c'] = dm_halos['halo_mass']/cosmo_ss.h # sod_halo_mass = M200,c in Msun/h, needs conversion

filt3 = ncdata_Table["M200c"] >= mmin
filt4 = ncdata_Table["M200c"] < mmax
filt5 = ncdata_Table["redshift_true"] >= zmin
filt6 = ncdata_Table["redshift_true"] < zmax

filt = filt3 * filt4 * filt5 * filt6

data_m_ss = ncdata_Table["M200c"][filt]  # M200,c [Msun]
data_z_ss = ncdata_Table["redshift_true"][filt]
N_cl = len(data_m_ss)
print(f"There are {N_cl} halos in this mass (Msod) and redshift range")

# Set up NumCosmo cosmology

In [ ]:
cosmo = Nc.HICosmoDEXcdm()
reion = Nc.HIReionCamb.new()
prim = Nc.HIPrimPowerLaw.new()

cosmo.add_submodel(reion)
cosmo.add_submodel(prim)

dist = Nc.Distance.new(2.0)

tf = Nc.TransferFunc.new_from_name("NcTransferFuncEH")

psml = Nc.PowspecMLTransfer.new(tf)

# psml = Nc.PowspecMLCBE.new ()
psml.require_kmin(1.0e-6)
psml.require_kmax(1.0e3)

psf = Ncm.PowspecFilter.new(psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()

In [ ]:
cosmo.props.H0 = cosmo_ss.H0.value
cosmo.props.Omegab = cosmo_ss.Ob0
cosmo.props.Omegac = cosmo_ss.Odm0
cosmo.props.Omegax = cosmo_ss.Ode0

cosmo.omega_x2omega_k()
cosmo.param_set_by_name("Omegak", 0.0)

prim.props.n_SA = cosmo_ss.n_s
print(cosmo_ss.sigma8, cosmo.sigma8(psf), cosmo.Omega_k0())

old_amplitude = math.exp(prim.props.ln10e10ASA)
prim.props.ln10e10ASA = math.log(
    (cosmo_ss.sigma8 / cosmo.sigma8(psf)) ** 2 * old_amplitude
)
print(cosmo_ss.sigma8, cosmo.sigma8(psf))

# Set up fit with NumCosmo

## Mass function, mass and redshift distributions

In [ ]:
#
# New multiplicity function 'NcMultiplicityFuncTinkerMean'
#
mulf = Nc.MultiplicityFuncTinker.new_full(Nc.MultiplicityFuncMassDef.CRITICAL, 200)
# mulf = Nc.MultiplicityFuncWatson.new ()
# mulf.set_mdef (Nc.MultiplicityFuncMassDef.FOF)


# 0 correspond to DM only and 1 to hydro sim
# mulf = Nc.MultiplicityFuncBocquet.new_full (Nc.MultiplicityFuncMassDef.CRITICAL,0,200)

#
# New mass function object using the objects defined above.
#
mf = Nc.HaloMassFunction.new(dist, psf, mulf)

# mf.set_area_sd (skysim_cat.sky_area) # Not sure if this is the way to setup a sky are

In [ ]:
#
# New Cluster Mass object
#
lnM_min = math.log(mmin)
lnM_max = math.log(mmax)


# using Log normal distribution
# cluster_m = Nc.ClusterMass.new_from_name ("NcClusterMassLnnormal{'lnMobs-min':<%20.15e>, 'lnMobs-max':<%20.15e>}" % (lnM_min, lnM_max))
# cluster_m.props.bias       = 0.0
# cluster_m.props.sigma      = 0.2
# print(cluster_m.lnMobs_min, cluster_m.lnMobs_max, lnM_min, lnM_max)

# no distribution - assumes masses are perfectly known
cluster_m = Nc.ClusterMass.new_from_name(
    "NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (lnM_min, lnM_max)
)
print(cluster_m.props.lnM_min, cluster_m.props.lnM_max, lnM_min, lnM_max)

In [ ]:
#
# New Cluster Redshift object
#

# using a global gaussian distribution
# cluster_z = Nc.ClusterRedshift.new_from_name ("NcClusterPhotozGaussGlobal{'pz-min':<%20.15e>, 'pz-max':<%20.15e>, 'z-bias':<0.0>, 'sigma0':<0.03>}" % (zmin, zmax))

# no distribution - assumes redshifts are perfectly known
cluster_z = Nc.ClusterRedshiftNodist(z_min=zmin, z_max=zmax)
print(cluster_z.props.z_min, cluster_z.props.z_max, zmin, zmax)

## Set up objects for the fit

In [ ]:
#
# New Cluster abundance object that uses all objects above
#
cad = Nc.ClusterAbundance.new(mf, None)

#
# New NcmData object for number count calculations
#
ncdata = Nc.DataClusterNCount.new(cad, "NcClusterRedshiftNodist", "NcClusterMassNodist")

#
#  Creating a new Modelset and set cosmo as the HICosmo model to be used
#  and cluster_m as the distribution of the mass-observable relation
#
mset = Ncm.MSet.new_array([cosmo, cluster_z, cluster_m])

In [ ]:
#
# Fill ncdata with SkySim masses and redshifts
#

ncdata.set_lnM_true(Ncm.Vector.new_array(np.log(data_m_ss)))
ncdata.set_z_true(Ncm.Vector.new_array(data_z_ss))

# Because we are using true masses and redshifts in this example,
# we replicate the true data in the 'observed' masses and redshift attributes
ncdata.set_lnM_obs(Ncm.Matrix.new_array(data_m_ss, 1))
ncdata.set_z_obs(Ncm.Matrix.new_array(data_z_ss, 1))
# ncdata.props.area = skysim_cat.sky_area
mf.set_area_sd(skysim_cat.sky_area)
ncdata.true_data(True)
ncdata.set_init(True)

# #
# # Save to a fits file
# #
# ncdata.catalog_save ("skysim%ld_data.fits" % ahash, True)

In [ ]:
# If the catalog already exists, skip previous step and load it directly

# ncdata.catalog_load("skysim5000_halos_m200c_13.fits")

## Define free parameters and prepare likelihood

In [ ]:
cosmo.props.Omegac_fit = True
cosmo.props.Omegab_fit = False
prim.props.ln10e10ASA_fit = True


# data set
dset = Ncm.Dataset.new()
dset.append_data(ncdata)


# New likelihood object using dset
lh = Ncm.Likelihood.new(dset)

## Fit for parameters (Omegac, ln10e10ASA) --> (Omegam, sigma8)

### Simple fit

In [ ]:
#
#  Creating a Fit object of type NLOPT using the fitting algorithm ln-neldermead to
#  fit the Modelset mset using the Likelihood lh and using a numerical differentiation
#  algorithm (NUMDIFF_FORWARD) to obtain the gradient (if needed).
#
fit = Ncm.Fit.new(
    Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_FORWARD
)

# fit.run_restart (Ncm.FitRunMsgs.SIMPLE,1e-3,0,None,None)

#
# Printing fitting informations.
#
fit.log_info()
fit.obs_fisher()
fit.log_covar()

In [ ]:
print(cosmo.props.Omegac, cosmo_ss.Odm0)
print(cosmo_ss.sigma8, cosmo.sigma8(psf))

### MCMC

In [ ]:
#
# Setting single thread calculation.
#
Ncm.func_eval_set_max_threads(150)
Ncm.func_eval_log_pool_stats()

#
# Additional functions as we want the chains for sigma8 and Omegam, which are derived parameters
#
mfunc_oa = Ncm.ObjArray.new()

mfunc_sigma8 = Ncm.MSetFuncList.new("NcHICosmo:sigma8", psf)
mfunc_Omegam = Ncm.MSetFuncList.new("NcHICosmo:Omega_m0", None)

mfunc_oa.add(mfunc_sigma8)
mfunc_oa.add(mfunc_Omegam)

print(mfunc_sigma8.eval0(mset))
print(mfunc_Omegam.eval0(mset))

#
# New Gaussian prior to provide the initial points for the chain.
# It was created with size 0 (number of parameters), but once
# initialized with mset the correct size is assigned.
#
# The initial sampler will use a diagonal covariance with the
# diagonal terms being the parameters scale set by each model.
#
init_sampler = Ncm.MSetTransKernGauss.new(0)
init_sampler.set_mset(mset)
init_sampler.set_prior_from_mset()
init_sampler.set_cov_from_rescale(1.0)  # 1

#
# Creates the ESMCMC walker object, this object is responsible
# for moving the walkers in each interation, the stretch move
# is affine invariant and therefore gives good results even for
# very correlated parametric space.
#
sampler = "apes"
# sampler  = 'stretch'
nwalkers = int(math.ceil(500))  # 500
ssize = 15000  # 1000000

if sampler == "apes":
    walker = Ncm.FitESMCMCWalkerAPES.new(nwalkers, mset.fparams_len())
elif sampler == "stretch":
    walker = Ncm.FitESMCMCWalkerStretch.new(nwalkers, mset.fparams_len())
#
# The methods below set the walk scale, which controls the size of the
# step done between two walkers and circumscribe the walkers inside
# the box defined by the parameters inside the mset object.
#
# walker.set_scale (3.0)
# walker.set_box_mset (mset)
#
# Initialize the ESMCMC object using the objects above. It will
# use 50 walkers, i.e., each point in the MCMC chain contains
# 50 points in the parametric space. Each step uses the last point
# in the chain (the last 50 parametric points) to calculate the
# proposal points.
#
esmcmc = Ncm.FitESMCMC.new_funcs_array(
    fit, nwalkers, init_sampler, walker, Ncm.FitRunMsgs.SIMPLE, mfunc_oa
)

#
# These methods enable the auto-trim options on ESMCMC. This option
# makes the sampler check the chains' health and trim any unnecessary
# burn-in part. We set the number of divisions to 100 so we test the
# chains in blocks of n/100. The last method asserts that each 2min
# the catalog will be checked.
#
# esmcmc.set_auto_trim (True)
# esmcmc.set_auto_trim_div (100)
# esmcmc.set_max_runs_time (2.0 * 60.0)
# esmcmc.set_nthreads (4)
esmcmc.set_data_file("z_0_1_M_14_15.fits")
esmcmc.set_nthreads(150)

#
# Running the esmcmc, it will first calculate 1000 points, after that
# it will estimate the error in the parameters mean. Using the current
# errors the algorithm tries to calculated how many extra steps are
# necessary to obtain the required error `10^-3' in every parameters,
# and it will run such extra steps. It will repeat this procedure
# until it attains the required error in every parameter.
#
#

esmcmc.start_run()
esmcmc.run(ssize / nwalkers)
# esmcmc.run (10)
# esmcmc.run_lre (50, 1.0e-3)
esmcmc.end_run()

#
# Calculates the parameter means and covariance and set it into
# the fit object and then print.
#
esmcmc.mean_covar()
fit.log_covar()

In [ ]:
mcat = esmcmc.peek_catalog()
print(cosmo_ss.Odm0 + cosmo_ss.Ob0, cosmo_ss.sigma8)
print(cosmo.props.Omegac + cosmo.props.Omegab, cosmo.sigma8(psf))
"""
ntests   = 100.0
nwalkers = 500
burnin   = 10
mcat = Ncm.MSetCatalog.new_from_file_ro ("z_0_1_M_14_15.fits", nwalkers * burnin)

mcat.log_current_chain_stats ()
mcat.calc_max_ess_time (ntests, Ncm.FitRunMsgs.SIMPLE);
mcat.calc_heidel_diag (ntests, 0.0, Ncm.FitRunMsgs.SIMPLE);

mset.pretty_log ()
mcat.log_full_covar ()
mcat.log_current_stats ()

be, post_lnnorm_sd = mcat.get_post_lnnorm ()
lnevol, glnvol = mcat.get_post_lnvol (0.6827)

Ncm.cfg_msg_sepa ()
print ("# Bayesian evidence:                                 % 22.15g +/- % 22.15g" % (be, post_lnnorm_sd))
print ("# 1 sigma posterior volume:                          % 22.15g" % lnevol)
print ("# 1 sigma posterior volume (Gaussian approximation): % 22.15g" % glnvol)
"""

In [ ]:
from chainconsumer import ChainConsumer

c = ChainConsumer()

nwalkers = mcat.nchains()
m2lnL = mcat.get_m2lnp_var()

rows = np.array(
    [mcat.peek_row(i).dup_array() for i in range(nwalkers * 20, mcat.len())]
)
params = ["$" + mcat.col_symb(i) + "$" for i in range(mcat.ncols())]
posterior = -0.5 * rows[:, m2lnL]

rows = np.delete(rows, m2lnL, 1)
params = np.delete(params, m2lnL, 0)

indices = [0, 1]

rows = rows[:, indices]
params = params[indices]

c.add_chain(rows, posterior=posterior, parameters=list(params))
c.configure(
    kde=True,
    label_font_size=11,
    sigma2d=False,
    sigmas=[1, 2, 3],
    spacing=0.0,
    tick_font_size=11,
    usetex=False,
)


plot_args = {}
plot_args["truth"] = [cosmo_ss.sigma8, cosmo_ss.Odm0 + cosmo_ss.Ob0]
plot_args["extents"] = [(0.79, 0.83), (0.24, 0.27)]

fig = c.plotter.plot(**plot_args)
c.plotter.plot(filename="z[0.0,1.0]_camb.png", figsize=0.75, **plot_args)
fig.set_size_inches(14.0, 14.0)

In [ ]:
ncdata.get_length()

In [ ]:
skysim_cat.sky_area

In [ ]:
# list(np.sort(skysim_cat.list_all_native_quantities ()))

In [ ]:
print(skysim_cat.get_quantity_info("hostHaloMass"))
print(skysim_cat.get_quantity_info("baseDC2/sod_halo_cdelta"))
print(skysim_cat.get_quantity_info("baseDC2/sod_halo_mass"))

In [ ]:
skysim_cat.halo_mass_def

In [ ]:
plt.figure(dpi=120)

zbmin = zmin
zbmax = zmax

# zbmin = 0.0
# zbmax = 0.5

ff = (data_z_ss >= zbmin) * (data_z_ss < zbmax)

n, bins, patches = plt.hist(
    np.log(data_m_ss[ff]), bins=70, range=(np.log(mmin), np.log(mmax))
)

a1 = []
a2 = []

m2lnL = 0.0

for bi, bi_1, nci in zip(bins[:-1], bins[1:], n):
    ni = mf.n(cosmo, bi, bi_1, zbmin, zbmax, Nc.HaloMassFunctionSplineOptimize.NONE)
    a1.append(0.5 * (bi + bi_1))
    a2.append(ni)
    m2lnL = m2lnL + 2.0 * (ni - nci * math.log(ni))

plt.plot(a1, a2, c="r")

Ntheo = mf.n(
    cosmo, lnM_min, lnM_max, zbmin, zbmax, Nc.HaloMassFunctionSplineOptimize.NONE
)
Ndata = ncdata.get_len()
print("FFIT", Ntheo, Ndata, m2lnL, math.sqrt(Ntheo))
cosmo.params_log_all()

Omegac_fit = cosmo.param_get_by_name("Omegac")
ln10e10ASA_fit = prim.props.ln10e10ASA

cosmo.param_set_by_name("Omegac", 0.22)
prim.props.ln10e10ASA = 3.0813

mf.prepare(cosmo)
cad.prepare(cosmo, cluster_z, cluster_m)

a1 = []
a2 = []

m2lnL = 0.0

for bi, bi_1, nci in zip(bins[:-1], bins[1:], n):
    ni = mf.n(cosmo, bi, bi_1, zbmin, zbmax, Nc.HaloMassFunctionSplineOptimize.NONE)
    a1.append(0.5 * (bi + bi_1))
    a2.append(ni)
    m2lnL = m2lnL + 2.0 * (ni - nci * math.log(ni))

plt.plot(a1, a2, c="k")
plt.ylabel("dn/dlnM")
plt.xlabel("lnM")
plt.xlim(np.log(mmin), np.log(mmax))

Ntheo = mf.n(
    cosmo, lnM_min, lnM_max, zbmin, zbmax, Nc.HaloMassFunctionSplineOptimize.NONE
)
Ndata = ncdata.get_len()
print("NFIT", Ntheo, Ndata, m2lnL, math.sqrt(Ntheo))
cosmo.params_log_all()

cosmo.param_set_by_name("Omegac", Omegac_fit)
prim.props.ln10e10ASA = ln10e10ASA_fit

mf.prepare(cosmo)
cad.prepare(cosmo, cluster_z, cluster_m)

In [ ]:
plt.figure(dpi=120)

# bunga = dm_halos['baseDC2/redshift'][filt]
# bunga = dm_halos['redshift'][filt]
bunga = data_z_ss

n, bins, patches = plt.hist(bunga, bins=70, range=(zmin, zmax))

a1 = []
a2 = []

m2lnL = 0.0

for bi, bi_1, nci in zip(bins[:-1], bins[1:], n):
    ni = mf.n(cosmo, lnM_min, lnM_max, bi, bi_1, Nc.HaloMassFunctionSplineOptimize.NONE)
    a1.append(0.5 * (bi + bi_1))
    a2.append(ni)
    m2lnL = m2lnL + 2.0 * (ni - nci * math.log(ni))


plt.plot(a1, a2, c="r")

Ntheo = mf.n(
    cosmo, lnM_min, lnM_max, zbmin, zbmax, Nc.HaloMassFunctionSplineOptimize.NONE
)
Ndata = ncdata.get_len()
print("FFIT", Ntheo, Ndata, m2lnL, math.sqrt(Ntheo))
cosmo.params_log_all()

Omegac_fit = cosmo.param_get_by_name("Omegac")
ln10e10ASA_fit = prim.props.ln10e10ASA

cosmo.param_set_by_name("Omegac", 0.22)
prim.props.ln10e10ASA = 3.0813

mf.prepare(cosmo)
cad.prepare(cosmo, cluster_z, cluster_m)

a1 = []
a2 = []

m2lnL = 0.0

for bi, bi_1, nci in zip(bins[:-1], bins[1:], n):
    ni = mf.n(cosmo, lnM_min, lnM_max, bi, bi_1, Nc.HaloMassFunctionSplineOptimize.NONE)
    a1.append(0.5 * (bi + bi_1))
    a2.append(ni)
    m2lnL = m2lnL + 2.0 * (ni - nci * math.log(ni))


plt.plot(a1, a2, c="k")
plt.ylabel("dn/dz")
plt.xlabel("z")

Ntheo = mf.n(
    cosmo, lnM_min, lnM_max, zbmin, zbmax, Nc.HaloMassFunctionSplineOptimize.NONE
)
Ndata = ncdata.get_len()
print("NFIT", Ntheo, Ndata, m2lnL, math.sqrt(Ntheo))
cosmo.params_log_all()

cosmo.param_set_by_name("Omegac", Omegac_fit)
prim.props.ln10e10ASA = ln10e10ASA_fit

mf.prepare(cosmo)
cad.prepare(cosmo, cluster_z, cluster_m)

In [ ]:
# sorted(skysim_cat.list_all_native_quantities ())

### 